<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Generative AI with OpenAI API</h1>
<h1>Embeddings</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from ipywidgets import interact
import tiktoken

import openai
from openai import OpenAI

import nltk
from nltk.corpus import reuters
from nltk import bigrams, trigrams

import scipy
from scipy.spatial.distance import cosine as cosine_similarity

import os
import gzip

import tqdm as tq
from tqdm.notebook import tqdm
tqdm.pandas()

import watermark

%load_ext watermark

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.12.3

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 3a7a9a8b6856eb5855cd2ac76a384e203382ab54

json     : 2.0.9
numpy    : 1.26.4
openai   : 1.30.5
scipy    : 1.11.4
tiktoken : 0.7.0
pandas   : 2.2.3
watermark: 2.4.3
tqdm     : 4.66.4
nltk     : 3.8.1



Get the API key

In [3]:
client = OpenAI()

# Extract embeddings

In [5]:
response = client.embeddings.create(
    model="text-embedding-ada-002",
    input="What is prompt engineering?",
    )

In [6]:
len(response.data)

1

The embedding itself is a high dimensional vector

In [7]:
response.data[0].embedding

[-0.013495800085365772,
 -0.0033809461165219545,
 -0.011480874381959438,
 -0.011949624866247177,
 0.004152284469455481,
 0.007041743025183678,
 -0.00880130473524332,
 -0.007870800793170929,
 -0.021996265277266502,
 -0.024514921009540558,
 0.013516788370907307,
 0.01193563174456358,
 -0.012257460504770279,
 0.010935165919363499,
 -0.009053169749677181,
 0.004593049641698599,
 0.020974809303879738,
 -0.001521688187494874,
 -0.0151539146900177,
 0.02707555517554283,
 -0.03414178639650345,
 -0.009605875238776207,
 0.008983206935226917,
 -0.009067162871360779,
 -0.017238803207874298,
 -0.01784048229455948,
 0.015419773757457733,
 -0.036268651485443115,
 -0.010851210914552212,
 0.0015076956478878856,
 -0.016930967569351196,
 -0.02297574281692505,
 -0.01789645105600357,
 -0.001575909205712378,
 -0.020457085222005844,
 0.0010074625024572015,
 -0.00024771157768554986,
 -0.004316696897149086,
 0.03044775500893593,
 0.008101677522063255,
 0.05118469148874283,
 0.017112869769334793,
 0.01037546433

In [8]:
len(response.data[0].embedding)

1536

And the similarity between vectors represents the similarity between the texts

In [9]:
emb1 = client.embeddings.create(
    model="text-embedding-ada-002",
    input="love",
    ).data[0].embedding

emb2 = client.embeddings.create(
    model="text-embedding-ada-002",
    input="kindness",
    ).data[0].embedding

Cosine similarity is a quick and dirty way to measure how similar two vectors are

In [10]:
cosine_similarity(emb1, emb2)

0.16235277591731323

Naturally, unrelated words have much lower degree of similarity

In [11]:
emb3 = client.embeddings.create(
    model="text-embedding-ada-002",
    input="airplane",
    ).data[0].embedding

In [12]:
cosine_similarity(emb2, emb3)

0.22998445414768232

# Generate embeddings for the Reuters sentences dataset

We're going to use Reuters

In [13]:
sentences = [" ".join(word for word in sent) for sent in reuters.sents()]

In [14]:
len(sentences)

54716

In [15]:
sentences[:10]

["ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said .",
 'They told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And lead to curbs on American imports of their products .',
 "But some exporters said that while the conflict would hurt them in the long - run , in the short - term Tokyo ' s loss might be their gain .",
 "The U . S . Has said it will impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to stick to a pact not to sell semiconductors on world markets at below cost .",
 'Unofficial Japanese estimates put the impact of the tariffs at 10 billion dlrs and spokesmen for major electronics firms said they would virtually halt expo

Let us build a DataFrame of 1,000 sentences

In [16]:
data = pd.DataFrame(sentences[:1000], columns=["text"])

In [17]:
data

,text
0,ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPA...
1,They told Reuter correspondents in Asian capit...
2,But some exporters said that while the conflic...
3,The U . S . Has said it will impose 300 mln dl...
4,Unofficial Japanese estimates put the impact o...
...,...
995,The decision to cut deposits was taken by the ...
996,The cuts were likely to attract more business ...
997,ELECTRO RENT CORP & lt ; ELRC > 3RD QTR FEB 28...
998,WALGREEN CO 2ND QTR SHR 62 CTS VS 58 CTS


In [18]:
embedding_model = "text-embedding-ada-002"

In [19]:
encoding = tiktoken.encoding_for_model(embedding_model)

# Compute the embeddings for each sentence

In [21]:
def get_embedding(text, engine):
    text = text.replace("\n", " ")
    
    return client.embeddings.create(input=[text], model=engine).data[0].embedding

In [22]:
data["embedding"] = data.text.progress_apply(lambda x: get_embedding(x, engine=embedding_model))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [23]:
data

,text,embedding
0,ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPA...,"[-0.016284245997667313, -0.028311172500252724,..."
1,They told Reuter correspondents in Asian capit...,"[-0.021919183433055878, -0.016359390690922737,..."
2,But some exporters said that while the conflic...,"[-0.020123226568102837, -0.01861906610429287, ..."
3,The U . S . Has said it will impose 300 mln dl...,"[-0.0199994258582592, -0.016706187278032303, 0..."
4,Unofficial Japanese estimates put the impact o...,"[-0.021230917423963547, -0.03368213027715683, ..."
...,...,...
995,The decision to cut deposits was taken by the ...,"[0.003791265655308962, -0.014937720261514187, ..."
996,The cuts were likely to attract more business ...,"[-0.020329460501670837, -0.02050635777413845, ..."
997,ELECTRO RENT CORP & lt ; ELRC > 3RD QTR FEB 28...,"[0.0015153614804148674, -0.012284890748560429,..."
998,WALGREEN CO 2ND QTR SHR 62 CTS VS 58 CTS,"[0.0023807904217392206, 0.006467325147241354, ..."


# Generate Recommendations

In [24]:
def get_recommendations(data, pos, k_nearest_neighbors=3):
    embeddings = np.array(data["embedding"].to_list())
    strings = data["text"].to_list()

    query_embedding = embeddings[pos]

    distances = [ cosine_similarity(query_embedding, embedding) 
                 for embedding in tqdm(embeddings, total=len(embeddings)) 
                ]
    
    indices_of_nearest_neighbors = np.argsort(distances)

    query_string = strings[pos]
    print(f"Query: {query_string}\n\n")

    for pos, i in enumerate(indices_of_nearest_neighbors[1:k_nearest_neighbors+1]):
        print("-- Nearest neighbor %u of %u (%u)---\n%s\n\n" % (pos, k_nearest_neighbors, i, strings[i]))

In [28]:
data.iloc[456].text

'" For the time being , we will have to focus our policy more on the external side , and we can live with a more expansionary money supply .'

In [29]:
%%time
get_recommendations(data, 456, 5)

  0%|          | 0/1000 [00:00<?, ?it/s]

Query: " For the time being , we will have to focus our policy more on the external side , and we can live with a more expansionary money supply .


-- Nearest neighbor 0 of 5 (448)---
Demands for greater stimulation of its domestic economy , accelerating tax cuts , cutting interest rates and tolerating above - target money supply growth .


-- Nearest neighbor 1 of 5 (186)---
He told a press conference the government planned to increase investment , speed up the opening of the local market to foreign imports and gradually adjust its currency to hold the surplus " at a proper level ."


-- Nearest neighbor 2 of 5 (453)---
Poehl said the Bundesbank had tolerated rapid money supply growth last year because the country enjoyed low inflation and because external factors , including low oil prices and favourable terms of trade , had given some extra leeway .


-- Nearest neighbor 3 of 5 (192)---
" Our government projections of eight pct GNP growth , five billion dlrs of ( current account ) 

<center>
     <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>